In [4]:
from tensorflow.keras.layers import TimeDistributed,LSTM,Conv1D,Dense,Flatten,ConvLSTM2D,MaxPooling1D, Dropout
# from tensorflow.tensorflow_model_optimization.sparsity import keras as sparsity
from sklearn.experimental import enable_iterative_imputer
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from sklearn.impute import IterativeImputer
from sklearn.model_selection import KFold 
from IPython.display import clear_output
from scipy.interpolate import interp1d
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow import keras
from scipy import stats 
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np
import math
import csv

In [6]:
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)

def norm(x, train_mean, train_std):
    return (x - train_mean) / train_std

In [7]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
features = ["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature] = train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    test[feature] = test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])

patterns =  len(train['temp'])
entries =  len(train['temp'][0])
num_features = len(features)
labels = np.array(train['target'])
count = 0
inputs = [0, 0, 0, 0, 0, 0]
for F in features:
    with open(F + '.txt', 'r') as f:
        l = ([[float(num) for num in line.split(' ')] for line in f])
    l = np.array(l)
    inputs[count] = l
    count += 1
    
temperature, precipitation, humidity, wind_d, wind_s, pressure = inputs[0], inputs[1], inputs[2], inputs[3], inputs[4], inputs[5]

# Locations to numbers
for i in range(patterns):
    if train['location'][i] == 'A':
        train['location'][i] = -1
    elif train['location'][i] == 'B':
        train['location'][i] = -0.5
    elif train['location'][i] == 'C':
        train['location'][i] = 0.05
    elif train['location'][i] == 'D':
        train['location'][i] = 0.5
    elif train['location'][i] == 'E':
        train['location'][i] = 1

vector = np.zeros((patterns, entries*(num_features + 1)))
for i in range(patterns): 
    vector[i,0:121],vector[i,121:242], vector[i,242:363], vector[i,363:484], vector[i,484:605], vector[i,605:726], vector[i,726:] = temperature[i], precipitation[i], humidity[i], wind_d[i], wind_s[i], pressure[i], np.ones(121)*train['location'][i]
clear_output()

vector_train, vector_test, labels_train, labels_test = train_test_split(vector, labels, test_size = 0.3, random_state = 777)

# Deep Feed Forward Neural Network

In [15]:
def FFNN(X, y, epochs, folds, batchsize, sim):
    "----------------------------------------------------------------------"
    "This function returns a trained and tested model"
    "model based on data x and labels y over epochs."

    "The input data X must be of a matrix or tensor size only"
    "With shape = [patterns, features, entries, depth], depth can be null"
    "and is assumed 1"
    "----------------------------------------------------------------------"
    with tf.device('/device:GPU:0'):
        kf = KFold(n_splits = folds, shuffle = True, random_state = 777)
    #     stop = EarlyStopping(monitor='val_loss', patience = 20, mode='min', verbose=0)
        training_loss_fold = np.zeros((folds, epochs))
        testing_loss_fold = np.zeros((folds, epochs))

        fold = -1
        for train_index, test_index in kf.split(X):
            clear_output()
            fold += 1
            print("Simulation {}\nFold {}".format(sim,fold))
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            # This standardizes the data.
            start = 0
            for i in range(7):
                end = (i + 1)*121
                m = np.mean(X_train[:,start:end]) #mean of training
                s = np.std(X_train[:,start:end]) #mean of testing
                X_train[:,start:end] = norm(X_train[:,start:end], m, s)
                X_test[:,start:end] = norm(X_test[:,start:end], m, s)
                start = end


            model = keras.Sequential()
            model.add(layers.Dense(764, activation='relu', input_shape=[847]))
            model.add(layers.Dropout(0.2))
            model.add(layers.BatchNormalization())
            model.add(layers.Dense(364, activation='relu'))
            model.add(layers.Dropout(0.2))
            model.add(layers.BatchNormalization())
            model.add(layers.Dense(264, activation='relu'))
            model.add(layers.Dropout(0.2))
            model.add(layers.BatchNormalization())
            model.add(layers.Dense(164, activation='relu'))
            model.add(layers.Dropout(0.2))
            model.add(layers.BatchNormalization())
            model.add(layers.Dense(64, activation='relu'))
            model.add(layers.Dense(1, activation = 'linear'))

            optimizer = tf.keras.optimizers.Adam()

            model.compile(loss='mse',
                        optimizer=optimizer,
                        metrics=['mae', 'mse'])

            history = model.fit(X_train, y_train, epochs = epochs, batch_size = batchsize, 
                                validation_data = (X_test, y_test))

            loss_train = np.sqrt(history.history['loss'])
            loss_test = np.sqrt(history.history['val_loss'])

            #Determining Best Model
            lowest_test_loss = np.min(loss_test)

            if fold == 0:
                model_best = model
                lowest_validation = lowest_test_loss
            elif fold > 0:
                if lowest_test_loss < lowest_validation:
                    model_best = model
                    lowest_validation = lowest_test_loss

            training_loss_fold[fold] = loss_train
            testing_loss_fold[fold] = loss_test

        return model_best, training_loss_fold, testing_loss_fold, lowest_validation

In [16]:
k = 5
n_simulations = 5
epochs = 200
batchsize = 64
sim_train = np.zeros((n_simulations, epochs))
sim_test = np.zeros((n_simulations, epochs))
simulation_list = []
with tf.device('/device:GPU:0'):
    for sim in range(n_simulations):
        model_sim_best, Ltrain, Ltest, lowest_val = FFNN(vector_train, labels_train, epochs, k, batchsize, sim)

        # Testing model
        m_label = np.mean(vector_test)
        s_label = np.std(vector_test)
        X_model = norm(vector_test, m_label, s_label)
        R = model_sim_best.evaluate(X_model, labels_test, verbose = 0)
        simulation_list.append(R[0])

        if sim == 0:
            model_best = model_sim_best
            val_marker = lowest_val
        else:
            if lowest_val < val_marker:
                model_best = model_sim_best
                val_marker = lowest_val

        sim_train[sim] = np.mean(Ltrain, axis = 0)
        sim_test[sim] = np.mean(Ltest, axis = 0)

        plt.figure(1)
        plt.plot(range(epochs), sim_train[sim], '--', label = 'Simulation {}'.format(sim+1))
        plt.figure(2)
        plt.plot(range(epochs), sim_test[sim], label = 'Simulation {}'.format(sim+1))


    plt.figure(3)    
    plt.plot(range(epochs), np.mean(sim_train, axis = 0), 'k--')
    plt.figure(4)    
    plt.plot(range(epochs), np.mean(sim_test, axis = 0), 'k')

    clear_output()

    titles = ['FFNN Simulation Training', 'FFNN Simulation Validation','FFNN Average Training', 'FFNN Average Validation']
    for i in range(4):
        plt.figure(i+1)
        plt.title(titles[i])
        plt.xlabel('Epochs')
        plt.ylabel('Root Mean Square Error')
        if i == 0 or i == 1:     
            plt.legend()   

    overfit = np.sqrt(simulation_list) - np.min(sim_train, axis = 1)        
    train_min_mean = np.mean(np.min(sim_train, axis = 1))
    train_min_std = np.std(np.min(sim_train, axis = 1))
    test_min_mean = np.mean(np.min(sim_test, axis = 1))
    test_min_std = np.std(np.min(sim_test, axis = 1))
    print('Final Training Mean RMSE:',train_min_mean)
    print('Final Training STD RMSE:',train_min_std)
    print('Final Validation Mean RMSE:',test_min_mean)
    print('Final Validation STD RMSE:',test_min_std)
    print('The Final Mean RMSE', np.mean(np.sqrt(simulation_list)))
    print('The Final STD RMSE', np.std(np.sqrt(simulation_list)))
    print('Overfit Mean', np.mean(overfit))
    print('Overfit STD', np.std(overfit))
    #-----------------Saving model------------------
    # serialize model to JSON
    model_json = model_best.to_json()
    with open("FFNN_model_best.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model_best.save_weights("FFNN_model_best.h5")
    print("Saved model to disk")

Simulation 0
Fold 1
Epoch 1/200
136/136 [==============================] - 3s 12ms/step - loss: 3996.8661 - mae: 47.5696 - mse: 3996.8661 - val_loss: 4418.9736 - val_mae: 53.6399 - val_mse: 4418.9736
Epoch 2/200
136/136 [==============================] - 1s 10ms/step - loss: 1173.2443 - mae: 23.0359 - mse: 1173.2443 - val_loss: 1623.4899 - val_mae: 29.1167 - val_mse: 1623.4899
Epoch 3/200
136/136 [==============================] - 1s 10ms/step - loss: 1033.0300 - mae: 21.5304 - mse: 1033.0300 - val_loss: 1073.8306 - val_mae: 22.7746 - val_mse: 1073.8306
Epoch 4/200
136/136 [==============================] - 1s 10ms/step - loss: 955.8752 - mae: 21.0219 - mse: 955.8752 - val_loss: 1088.7395 - val_mae: 22.8790 - val_mse: 1088.7395
Epoch 5/200
136/136 [==============================] - 1s 11ms/step - loss: 863.6479 - mae: 20.0561 - mse: 863.6479 - val_loss: 1419.6246 - val_mae: 25.4977 - val_mse: 1419.6246
Epoch 6/200
136/136 [==============================] - 1s 10ms/step - loss: 874.0026

KeyboardInterrupt: 

In [56]:
def prune(model):
    epochs_prune = 1
    batch = 1000
    end_step = np.ceil(1.0 * len(vector_train)/batch).astype(np.int32) * epochs_prune
    new_pruning_params = {'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity = 0.25,
                                                       final_sparsity = 0.25,
                                                       begin_step = 0,
                                                       end_step = end_step,
                                                       frequency = 100)}

    new_pruned_model = sparsity.prune_low_magnitude(model, **new_pruning_params)
    new_pruned_model.compile(loss = tf.keras.losses.mean_squared_error, optimizer = 'adam', metrics=['mse'])
    callbacks = [sparsity.UpdatePruningStep()]
    new_pruned_model.fit(vector_train, labels_train,
              batch_size = batch,
              epochs = epochs_prune,
              verbose = 1,
              callbacks = callbacks)
    #           validation_data = (x_val, y_val))

    clear_output()
    final_model = sparsity.strip_pruning(new_pruned_model)
    score = new_pruned_model.evaluate(vector_test, labels_test, verbose = 0)
    print('initial test RMSE = {:.2f}'.format(np.mean(np.sqrt(simulation_list))))
    print('The final test RMSE on the model is {:.2f}'.format(np.sqrt(score[0])))
    
    return new_pruned_model

prune(model_best)

initial test RMSE = 91.97
The final test RMSE on the model is 220.29


In [12]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [13]:
with tf.device('/device:GPU:0'):
    print('Working')

Working


In [33]:
import numpy as np
import tensorflow as tf
from datetime import datetime

# Choose which device you want to test on: either 'cpu' or 'gpu'
devices = ['cpu', 'gpu']

# Choose size of the matrix to be used.
# Make it bigger to see bigger benefits of parallel computation
shapes = [(50, 50), (100, 100), (500, 500), (10000, 10000)]


def compute_operations(device, shape):
    """Run a simple set of operations on a matrix of given shape on given device

    Parameters
    ----------
    device : the type of device to use, either 'cpu' or 'gpu' 
    shape : a tuple for the shape of a 2d tensor, e.g. (10, 10)

    Returns
    -------
    out : results of the operations as the time taken
    """
    
    # Define operations to be computed on selected device
    with tf.device(device):
        random_matrix = tf.random.uniform(shape=shape, minval=0, maxval=1)
        dot_operation = tf.matmul(random_matrix, tf.transpose(random_matrix))
        sum_operation = tf.reduce_sum(dot_operation)
#     print(sum_operation)
    # Time the actual runtime of the operations
    tf.compat.v1.disable_eager_execution()
    start_time = datetime.now()
#     with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True)) as session:
#             result = session.run(dot_operation)
    sess = tf.compat.v1.Session()
    result = sess.run(sum_operation)
    elapsed_time = datetime.now() - start_time

    return result, elapsed_time



if __name__ == '__main__':

    # Run the computations and print summary of each run
    for device in devices:
        print("--" * 20)

        for shape in shapes:
            _, time_taken = compute_operations(device, shape)

            # Print the result and also the time taken on the selected device
            print("Input shape:", shape, "using Device:", device, "took: {:.2f}".format(time_taken.seconds + time_taken.microseconds/1e6))
            #print("Computation on shape:", shape, "using Device:", device, "took:")

    print("--" * 20)

----------------------------------------
Input shape: (50, 50) using Device: cpu took: 0.05
Input shape: (100, 100) using Device: cpu took: 0.04
Input shape: (500, 500) using Device: cpu took: 0.04
Input shape: (10000, 10000) using Device: cpu took: 4.55
----------------------------------------
Input shape: (50, 50) using Device: gpu took: 0.05
Input shape: (100, 100) using Device: gpu took: 0.04
Input shape: (500, 500) using Device: gpu took: 0.04
Input shape: (10000, 10000) using Device: gpu took: 0.92
----------------------------------------


In [ ]:
tf.